In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
cd /content/drive/My\ Drive/CPM_Pytorch

/content/drive/My Drive/CPM_Pytorch


In [78]:
import matplotlib
matplotlib.use('TKAgg')

from data_loader.dataLoader import CarDataset as Mydata
from model.cpm import CPM
from src.util import heatmap_image,save_images,PCK

import os
import glob

import torch
import torch.optim as optim
import torch.nn as nn
import configparser
import pandas as pd
import numpy as np

import cv2

from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision import transforms
from PIL import Image, ImageDraw
from src.compute_origin import HumanSortKey,GetIntrinsicMatrix,DrawOrigin

In [10]:
#Hyperparams
config = configparser.ConfigParser()
config.read('conf_online.text')

train_data_dir = config.get('data', 'train_data_dir')
model_save_dir=config.get('data', 'model_save_dir')
annotation_save_dir=config.get('data', 'annotation_save_dir')
keypoints_dir=config.get('data','keypoints_file_dir')

save_dir = config.get('data', 'save_dir')

learning_rate = config.getfloat('training', 'learning_rate')
batch_size = config.getint('training', 'batch_size')
epochs = config.getint('training', 'epochs')
begin_epoch = config.getint('training', 'begin_epoch')
n_keypoints=config.getint('training', 'n_keypoints')  



In [11]:
cuda = torch.cuda.is_available()
 

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [12]:
# *********************** Build dataset ***********************
train_data = Mydata(data_dir=train_data_dir,n_keypoints=4)


print ('Train dataset total number of images sequence is ----' + str(len(train_data)))

# Data Loader
train_dataset = DataLoader(train_data, batch_size=batch_size, shuffle=True)

Train dataset total number of images sequence is ----170


In [13]:
train_data.images_dir.sort(key=HumanSortKey)
train_data.label_dir.sort(key=HumanSortKey)
train_data.label_dir[0:10]
begin_epoch

100

In [14]:
net = CPM(out_c=n_keypoints)

device_ids = [0]  
if cuda:
    net = net.cuda(device_ids[0])
    net = nn.DataParallel(net, device_ids=device_ids)
print(model_save_dir)
if begin_epoch > 0:
    save_path =model_save_dir+'model_epoch' + str(begin_epoch) + '.pth'
    state_dict = torch.load(save_path)
    net.load_state_dict(state_dict)

models/


In [15]:
def train():
    # *********************** initialize optimizer ***********************
    optimizer = optim.Adam(params=net.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    criterion = nn.MSELoss(size_average=True)                       # loss function MSE average

    net.train()
    for epoch in range(begin_epoch, epochs + 1):
        print ('epoch....................' + str(epoch))
        
        for step, (image, label_map, center_map, imgs) in enumerate(train_dataset):
            
            image = Variable(image.cuda() if cuda else image)                   # 4D Tensor
            # Batch_size  *  3  *  width(368)  *  height(368)

            # 4D Tensor to 5D Tensor
            label_map = torch.stack([label_map]*6, dim=1)
            # Batch_size  *  21 *   45  *  45
            # Batch_size  *   6 *   21  *  45  *  45
            label_map = Variable(label_map.cuda() if cuda else label_map)

            center_map = Variable(center_map.cuda() if cuda else center_map)    # 4D Tensor
            # Batch_size  *  width(368) * height(368)

            optimizer.zero_grad()
            pred_6 = net(image, center_map)  # 5D tensor:  batch size * stages * 21 * 45 * 45 #This is huge

            # ******************** calculate loss of each joints ********************
            loss = criterion(pred_6, label_map)

            # backward
            loss.backward()
            loss=loss.detach()
            #import ipdb
            #ipdb.set_trace()
            
            optimizer.step() #Not using SGD
            #gc.collect()

            if step % 10 == 0:
                print ('--step .....' + str(step))
                print ('--loss ',loss.item()*10000)
                gb_factor=10**9
                print('--gpu memory=',torch.cuda.memory_allocated()/gb_factor)

            #if step % 40 == 0:
              #torch.save(net.state_dict(), os.path.join(save_dir, 'model_epoch{:d}.pth'.format(epoch)))
            #    save_images(label_map[:, 5, :, :, :], pred_6[:, 5, :, :, :], step, epoch, imgs)

        if epoch % 50 == 0:
            torch.save(net.state_dict(), os.path.join(save_dir, 'model_epoch{:d}.pth'.format(epoch)))
        
            #Delete tensors from memory
            #del image,label_map,center_map,pred_6

    print ('train done!')

In [ ]:
train()

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


epoch....................0
--step .....0
--loss  17.512402264401317
--gpu memory= 1.592381952
--step .....10
--loss  16.61937334574759
--gpu memory= 1.592673792
--step .....20
--loss  15.87017672136426
--gpu memory= 1.592673792
--step .....30
--loss  15.672702575102448
--gpu memory= 1.592673792
--step .....40
--loss  15.222501242533326
--gpu memory= 1.592673792
epoch....................1
--step .....0
--loss  15.176802407950163
--gpu memory= 1.592534528
--step .....10
--loss  14.892352046445012
--gpu memory= 1.592534528
--step .....20
--loss  14.730667462572455
--gpu memory= 1.592534528
--step .....30
--loss  14.573261141777039
--gpu memory= 1.592534528
--step .....40
--loss  14.68525268137455
--gpu memory= 1.592534528
epoch....................2
--step .....0
--loss  14.54683137126267
--gpu memory= 1.592673792
--step .....10
--loss  14.363449299708009
--gpu memory= 1.592673792
--step .....20
--loss  14.05451213940978
--gpu memory= 1.592673792
--step .....30
--loss  14.398762723430991
-

KeyboardInterrupt: ignored

In [53]:
def get_locations(pred_6):
  locations=[]
  label_locations=[]
  for keypoint_map in pred_6:
    
    flat_tensor=keypoint_map.view(1,-1)
    val=torch.argmax(flat_tensor).cpu().numpy()
    locations.append(np.array([val//45*(368/45),(val%45)*(368/45)]) ) #return values corresponding to a 368x368 image
  return np.array(locations)  #x y locations of the heatmap


In [54]:
def generate_output_image(output,locations):
  
  output_image=transforms.ToPILImage()(output.cpu()).resize((368,368))
  draw = ImageDraw.Draw(output_image)
  for x,y in locations:
    draw.ellipse((y, x, y+5, x+5), fill = 'blue', outline ='blue')
  output_image.putalpha(150)
  return output_image

#output_image=generate_output_image(output.clone(),locations)
#output_image

In [86]:
#Read the 3d keypoints
keypoints_3d=pd.read_csv(keypoints_dir)

usable_object_points=keypoints_3d.to_numpy()[0:n_keypoints] #Only use the first n_keypoints keypoints

def GetFOV(train_data_dir,i):
  #returns fov in degrees
  fov_dir=sorted(glob.glob(train_data_dir+"FOV_Labels/*.txt"),key=HumanSortKey)
  fov_file=pd.read_csv(fov_dir[i]) #This value is in degrees
  fov=fov_file['FOV'][0] #FOV in degrees
  return fov

#Function to save a prediction from an image
def predict_output(net,train_data,i):
  image,labels,center_map,img_str=train_data[i]
  pred_6=net(torch.unsqueeze(image,dim=0),torch.unsqueeze(center_map,dim=0))[0][-1] #Final stage output
  locations=get_locations(pred_6.detach()) #Get locations from the heatmap
  
  locations_original_size=900/368*locations #Convert from 368x368 to 900x900, since the image labels are in the same order too
  fov=GetFOV(train_data_dir,i)

  empty_image=np.zeros((900,900,3))
  camera_matrix=GetIntrinsicMatrix(empty_image,fov) #This is the same regardless of whether we use labels or predictions

  ret_val,rvec_pred,tvec_pred=cv2.solvePnP(objectPoints=usable_object_points,imagePoints=locations_original_size,cameraMatrix=camera_matrix
                                             ,distCoeffs=np.zeros((4,1)))
  
  usable_labels=pd.read_csv(train_data.label_dir[i],delimiter=" ").to_numpy()[0:n_keypoints]
  ret_val,rvec_gt,tvec_gt=cv2.solvePnP(objectPoints=usable_object_points,imagePoints=usable_labels,cameraMatrix=camera_matrix
                                             ,distCoeffs=np.zeros((4,1)))
  
  print(locations_original_size)

  print(usable_labels)

  return tvec_gt-tvec_pred

predict_output(net,train_data,6)

[[520. 440.]
 [540. 520.]
 [580. 420.]
 [560. 500.]]
[[406.6622 502.2599]
 [472.4821 496.8001]
 [407.5432 562.2761]
 [504.0813 551.3449]]


array([[-260.56275795],
       [ 120.42171566],
       [ -31.19751093]])

In [87]:
#Prediction
i=0
for image,labels,center_map,img_str in train_data:
  pred_6=net(torch.unsqueeze(image,dim=0),torch.unsqueeze(center_map,dim=0))[0][-1] #Final stage output
  output=pred_6.clone().detach()*0
  pil_image=transforms.ToPILImage()(image)
  background=pil_image

  output_image=generate_output_image(output,get_locations(pred_6))
  background.paste(output_image,(0,0),output_image)
  background.save(annotation_save_dir+'Image'+str(i)+'.PNG')
  i+=1
  #import ipdb
  #ipdb.set_trace()
  print("Saving Image ",i)

Saving Image  1
Saving Image  2
Saving Image  3
Saving Image  4
Saving Image  5
Saving Image  6
Saving Image  7
Saving Image  8
Saving Image  9
Saving Image  10
Saving Image  11
Saving Image  12
Saving Image  13
Saving Image  14
Saving Image  15
Saving Image  16
Saving Image  17
Saving Image  18
Saving Image  19
Saving Image  20
Saving Image  21
Saving Image  22
Saving Image  23
Saving Image  24
Saving Image  25
Saving Image  26
Saving Image  27
Saving Image  28
Saving Image  29
Saving Image  30
Saving Image  31
Saving Image  32
Saving Image  33
Saving Image  34
Saving Image  35
Saving Image  36
Saving Image  37
Saving Image  38
Saving Image  39
Saving Image  40
Saving Image  41
Saving Image  42
Saving Image  43
Saving Image  44
Saving Image  45
Saving Image  46
Saving Image  47
Saving Image  48
Saving Image  49
Saving Image  50
Saving Image  51
Saving Image  52
Saving Image  53
Saving Image  54
Saving Image  55
Saving Image  56
Saving Image  57
Saving Image  58
Saving Image  59
Saving

In [63]:
train_data[0]

(tensor([[[0.3922, 0.3882, 0.3882,  ..., 0.2353, 0.2314, 0.2353],
          [0.3922, 0.3882, 0.3922,  ..., 0.2353, 0.2353, 0.2392],
          [0.3882, 0.3922, 0.3882,  ..., 0.2353, 0.2392, 0.2353],
          ...,
          [0.1020, 0.0863, 0.0667,  ..., 0.4510, 0.4196, 0.3961],
          [0.0863, 0.0588, 0.0510,  ..., 0.4353, 0.4235, 0.4078],
          [0.0667, 0.0510, 0.0549,  ..., 0.4118, 0.4314, 0.4157]],
 
         [[0.4588, 0.4549, 0.4510,  ..., 0.3098, 0.3098, 0.3098],
          [0.4588, 0.4549, 0.4549,  ..., 0.3098, 0.3098, 0.3098],
          [0.4549, 0.4588, 0.4549,  ..., 0.3098, 0.3098, 0.3137],
          ...,
          [0.1059, 0.0902, 0.0667,  ..., 0.3882, 0.3569, 0.3451],
          [0.0863, 0.0588, 0.0549,  ..., 0.3804, 0.3569, 0.3451],
          [0.0706, 0.0549, 0.0588,  ..., 0.3529, 0.3686, 0.3490]],
 
         [[0.6196, 0.6157, 0.6118,  ..., 0.4824, 0.4824, 0.4824],
          [0.6196, 0.6157, 0.6157,  ..., 0.4863, 0.4863, 0.4863],
          [0.6157, 0.6196, 0.6157,  ...,

In [ ]:
golf_annotation_save_dir='/content/drive/My Drive/Final Data/golf/Annotated_Images/'
golf_train_data_dir='/content/drive/My Drive/Final Data/golf/'

golf_train_data = Mydata(data_dir=golf_train_data_dir,n_keypoints=4)

golf_train_data.images_dir.sort(key=HumanSortKey)
golf_train_data.label_dir.sort(key=HumanSortKey)

#Predict on Golf using network trained on Mustang
i=0
for image,labels,center_map,img_str in golf_train_data:
  pred_6=net(torch.unsqueeze(image,dim=0),torch.unsqueeze(center_map,dim=0))[0][-1] #Final stage output
  output=pred_6.clone().detach()*0
  pil_image=transforms.ToPILImage()(image)
  background=pil_image

  output_image=generate_output_image(output,get_locations(pred_6))
  background.paste(output_image,(0,0),output_image)
  background.save(golf_annotation_save_dir+'Image'+str(i)+'.PNG')
  i+=1
  #import ipdb
  #ipdb.set_trace()
  print("Saving Image ",i)

Saving Image  1
Saving Image  2
Saving Image  3
Saving Image  4
Saving Image  5
Saving Image  6
Saving Image  7
Saving Image  8
Saving Image  9
Saving Image  10
Saving Image  11
Saving Image  12
Saving Image  13
Saving Image  14
Saving Image  15
Saving Image  16
Saving Image  17
Saving Image  18
Saving Image  19
Saving Image  20
Saving Image  21
Saving Image  22
Saving Image  23
Saving Image  24
Saving Image  25
Saving Image  26
Saving Image  27
Saving Image  28
Saving Image  29
Saving Image  30
Saving Image  31
Saving Image  32
Saving Image  33
Saving Image  34
Saving Image  35
Saving Image  36
Saving Image  37
Saving Image  38
Saving Image  39
Saving Image  40
Saving Image  41
Saving Image  42
Saving Image  43
Saving Image  44
Saving Image  45
Saving Image  46
Saving Image  47
Saving Image  48
Saving Image  49
Saving Image  50
Saving Image  51
Saving Image  52
Saving Image  53
Saving Image  54
Saving Image  55
Saving Image  56
Saving Image  57
Saving Image  58
Saving Image  59
Saving

['/content/drive/My Drive/Final Data/golf/Labels/labels1.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels7.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels3.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels2.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels4.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels5.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels6.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels8.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels13.txt',
 '/content/drive/My Drive/Final Data/golf/Labels/labels12.txt']